In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1

In [2]:
!pip install sentence-transformers text-generation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=b4f0f472da043521c01f4dae142380b1015b7c6ac205dcbab9795bc6d17b0d52
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from text_generation import Client

In [4]:
df = pd.read_csv("/content/shirts.csv")
df.head()

,Name,Description,Weight (kg),Regular price,Sale price,Categories,Size,Color,Brand
0,District® Very Important Tee®. DT6000,Crafted from ring spun cotton for ultimate com...,0.91,17.91,12.537,Sweatshirts/Fleece,XS,Athletic Heather,Port & Company
1,District® Very Important Tee®. DT6000,Crafted from ring spun cotton for ultimate com...,0.95,17.91,12.537,Sweatshirts/Fleece,S,Athletic Heather,Port & Company
2,District® Very Important Tee®. DT6000,Crafted from ring spun cotton for ultimate com...,1.03,17.91,12.537,Sweatshirts/Fleece,M,Athletic Heather,Port & Company
3,District® Very Important Tee®. DT6000,Crafted from ring spun cotton for ultimate com...,1.14,17.91,12.537,Sweatshirts/Fleece,L,Athletic Heather,Port & Company
4,District® Very Important Tee®. DT6000,Crafted from ring spun cotton for ultimate com...,1.18,17.91,12.537,Sweatshirts/Fleece,XL,Athletic Heather,Port & Company


In [5]:
from string import Template

cols = df.columns.to_list()
preamble ='Here is the details of a shirt in terms of  description, weight in (kg), regular price, sale price, categories, size, color and brand repsectively...\n'
prompt = "Use the following pieces of context to answer the question."
    # "If you don't know the answer, just say that you don't know, don't try to make up an answer."
template = Template(
    '$prompt\n'\
    'Description  : $description\n'\
    'Weight (kg)  : $weight\n'\
    'Regular price: $regular_price\n'\
    'Sale price   : $sale_price\n'\
    'Categoreis   : $categories\n'\
    'Size         : $size\n'\
    'Color        : $color\n'\
    'Brand        : $brand.<end>'
)
def format_input(row):
    params = {k:row[k] for k in cols}
    input_text = template.substitute(
        # preamble = preamble,
        prompt = prompt,
        description = params['Description'],
        weight = params["Weight (kg)"],
        regular_price = params['Regular price'],
        sale_price = params['Sale price'],
        categories = params['Categories'],
        size = params['Size'] ,
        color = params['Color'],
        brand = params['Brand'],
    )
    return input_text



In [6]:
df['input'] = df.apply(lambda x: format_input(x), axis=1)
df.head()

,Name,Description,Weight (kg),Regular price,Sale price,Categories,Size,Color,Brand,input
0,District® Very Important Tee®. DT6000,Crafted from ring spun cotton for ultimate com...,0.91,17.91,12.537,Sweatshirts/Fleece,XS,Athletic Heather,Port & Company,Use the following pieces of context to answer ...
1,District® Very Important Tee®. DT6000,Crafted from ring spun cotton for ultimate com...,0.95,17.91,12.537,Sweatshirts/Fleece,S,Athletic Heather,Port & Company,Use the following pieces of context to answer ...
2,District® Very Important Tee®. DT6000,Crafted from ring spun cotton for ultimate com...,1.03,17.91,12.537,Sweatshirts/Fleece,M,Athletic Heather,Port & Company,Use the following pieces of context to answer ...
3,District® Very Important Tee®. DT6000,Crafted from ring spun cotton for ultimate com...,1.14,17.91,12.537,Sweatshirts/Fleece,L,Athletic Heather,Port & Company,Use the following pieces of context to answer ...
4,District® Very Important Tee®. DT6000,Crafted from ring spun cotton for ultimate com...,1.18,17.91,12.537,Sweatshirts/Fleece,XL,Athletic Heather,Port & Company,Use the following pieces of context to answer ...


In [8]:
print(
    "".join((str(e) for e in df.input[:2])).split("<end>")[:-1].__len__()
    )

2


In [9]:
import argparse
from sentence_transformers import SentenceTransformer, CrossEncoder, util

from text_generation import Client
def embed():
    text = "".join(str(e) for e in df['input'])
    # text = " ".join(text.split())

    paragraphs = "".join((str(e) for e in df.input )).split("<end>")[:-1]
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    model.max_seq_length = 1024
    cross_encoder = CrossEncoder("cross-encoder/nli-deberta-v3-large")

    embeddings = model.encode(
        paragraphs,
        show_progress_bar=True,
        convert_to_tensor=True,
    )
    return model, cross_encoder, embeddings, paragraphs

In [22]:
def search(query, model, cross_encoder, embeddings, paragraphs, top_k):
    query_embeddings = model.encode(query, convert_to_tensor=True)
    # query_embeddings = query_embeddings.cuda()
    hits = util.semantic_search(
        query_embeddings,
        embeddings,
        top_k=top_k,
    )[0]
    cross_input = [[query, paragraphs[hit["corpus_id"]]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_input)

    for idx in range(len(cross_scores)):
        hits[idx]["cross_score"] = cross_scores[idx]

    print(hits)
    # Define a function to compute the L2 norm of 'cross_score' array
    def compute_l2_norm(item):
        return np.linalg.norm(item['cross_score'])

    results = []
    hits = sorted(hits, key=lambda x: np.max(x['cross_score']), reverse=True)
    for hit in hits[:5]:
        results.append(paragraphs[hit["corpus_id"]].replace("\n", " "))
    return results

In [11]:
from text_generation import Client
END_7B = "\n<|prompter|>{query}<|endoftext|><|assistant|>"
END_40B = "\nUser: {query}\nFalcon:"

PARAMETERS = {
    "temperature": 0.9,
    "top_p": 0.95,
    "repetition_penalty": 1.2,
    "top_k": 50,
    "max_new_tokens": 1024,
    }


In [12]:
model, cross_encoder, embeddings, paragraphs = embed()

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [13]:
from transformers.models.mt5.modeling_tf_mt5 import TFT5ForConditionalGeneration
import torch
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    T5ForConditionalGeneration,
    TFT5ForConditionalGeneration)

model_name = "google/flan-t5-base"

llm = T5ForConditionalGeneration.from_pretrained(
    model_name,
    # quantization_config=bnb_config,
    # trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True , truncation=True, padding = "longest")
# tokenizer.pad_token = tokenizer.eos_token

In [35]:
pre_prompt = "Below are a series of description of shirts from various brands.\
As an AI assistant. try to be helpful, polite, honest, sophisticated, emotionally aware, and humble-but-knowledgeable. \
It also tries to avoid giving false or misleading information, and it caveats when it isn't entirely sure about the right answer. That said, the assistant is practical and really does its best, and doesn't let caution get too much in the way of being useful.\n"

ending = "Provide the info in the given format."
query  = "suggest me a good shirt"
# query = input("Ask the bot: ")
results = search(
            query,
            model,
            cross_encoder,
            embeddings,
            paragraphs,
            top_k=5,
        )[0]
# print(results)


inputs = tokenizer(query+ results , return_tensors = "pt" , padding= "longest" , truncation = True)
response = tokenizer.batch_decode(
    llm.generate(**inputs,
                 num_beams = 1,
                 do_sample = True,
                 max_new_tokens = 1024,
                #  repetition_penalty = 20.0

                 ) ,
    skip_special_tokens = True,
    )[0]
print("Bot: ", response)

[{'corpus_id': 621, 'score': 0.4056379795074463, 'cross_score': array([-2.6493442 ,  2.3981235 , -0.12018439], dtype=float32)}, {'corpus_id': 622, 'score': 0.40550801157951355, 'cross_score': array([-2.698914  ,  2.4710503 , -0.13231538], dtype=float32)}, {'corpus_id': 623, 'score': 0.4054454565048218, 'cross_score': array([-2.619998  ,  2.3888948 , -0.13030134], dtype=float32)}, {'corpus_id': 610, 'score': 0.404449462890625, 'cross_score': array([-2.6920738 ,  2.5113378 , -0.19037516], dtype=float32)}, {'corpus_id': 612, 'score': 0.4032365083694458, 'cross_score': array([-2.6771574 ,  2.387784  , -0.09218742], dtype=float32)}]
Bot:  The relevant information to answer the above question is: This tee rocks with comfort. 4.5-ounce 100% soft spun cotton 30 singles50/50 soft spun cotton/poly (Neon and Heathered Charcoal)90/10 soft spun cotton/poly (Heather Grey)Form fitting 1x1 rib knit neckTear-away label Shoulder to shoulder tapingSize up for a less fitted lookDate:2012/09/27 7 o 11:56


In [66]:
df.Brand.unique()

array(['Port & Company', 'AllMade', 'Brooks Brothers', 'Carhartt'],
      dtype=object)